##### Dependencies
____

In [163]:
import numpy as np
import pandas as pd
import sqlalchemy as db
import datetime
import random
import hashlib

from faker import Faker

##### Database connection
____

In [164]:
g_params = {
    'dbuser'   : 'csc4710',
    'dbpwd'    : 'password',
    'dbserver' : 'localhost',
    'dbport'   : 3306,
    'dbname'   : 'CSC4710'
}

disk_engine = db.create_engine(f'mysql+pymysql://{g_params["dbuser"]}:{g_params["dbpwd"]}@{g_params["dbserver"]}:{g_params["dbport"]}/{g_params["dbname"]}', echo=False)
disk_engine.connect().connection.ping()

##### Load Credentials Dataframe
____

In [165]:
credentials = pd.read_sql_table(table_name='credentials', con=disk_engine)

cond = credentials['role'] == 'customer'
credentials = credentials[cond]

credentials.head(5)

,userid,password,employee_id,driver_id,customer_id,role
40,41,7ce3b56061ddd4f83e00525716e9936612bf444043e2a9...,NaN,NaN,1.0,customer
41,42,73bebcd884dd3f37688653d3cf8fa1b2ca1df0c46e3a23...,NaN,NaN,2.0,customer
42,43,64ee9f8e395826cae77a040e2cf7212022dbed3a059181...,NaN,NaN,3.0,customer
43,44,8d804083bee2e6986712060cc03fec99c43d75b62b2cf4...,NaN,NaN,4.0,customer
44,45,c6447eed39a98edd3e21e01316076b5c952c8e198809ac...,NaN,NaN,5.0,customer


##### Load Driver Dataframe
____

In [166]:
drivers = pd.read_sql_table(table_name='driver', con=disk_engine)

drivers.head(5)

,id,firstname,lastname,email,phone
0,1,Christopher,Patel,calvin11@hotmail.com,870-209-1746x49189
1,2,Alexis,Benson,rodriguezbrittney@gmail.com,(721)083-4901x665
2,3,Molly,Young,scole@hotmail.com,0434895465
3,4,Nicole,Lane,john43@hotmail.com,+1-555-378-4422x1679
4,5,Mary,Harris,sullivankristen@reyes.info,213-802-0221x06874


##### Load Address Dataframe
____

In [167]:
address = pd.read_sql_table(table_name='address', con=disk_engine)

address.head(5)

,id,street,city,state,zipcode,latitude,longitude
0,1,2175 PIEDMONT RD NE,Atlanta,GA,30324-4128,33.817098,-84.366785
1,2,306 LUCKIE ST NW,Atlanta,GA,30313-1706,33.762858,-84.395977
2,3,227 WELLINGTON ST SW,Atlanta,GA,30314-2240,33.749008,-84.438294
3,4,1125 CASCADE CIR SW @ZONE 4,Atlanta,GA,30311-2817,33.724948,-84.449931
4,5,469 FAIR ST SW,Atlanta,GA,30313-1203,33.748725,-84.403206


##### Load Customer Dataframe
____

In [168]:
customers = pd.read_sql_table(table_name='customer', con=disk_engine)

customers.head(5)

,id,firstname,lastname,email,phone,address
0,1,Kenneth,King,brandy02@heath.org,0529123086,32
1,2,Walter,Tanner,szamora@yahoo.com,(049)169-6111,19
2,3,Geoffrey,Nelson,moorescott@powell-reyes.net,001-019-776-6710x01150,42
3,4,Adrian,Hill,lindsaycampbell@gmail.com,(854)484-0320x861,6
4,5,Angela,Moon,ksavage@hotmail.com,+1-188-250-1479x06039,53


##### Load Payment Method Dataframe
____

In [169]:
payment_method = pd.read_sql_table(table_name='payment_method', con=disk_engine)

payment_method.head(5)

,id,userid,credit_card_id,paypal_id,google_pay_id
0,2,1,None,1.0,NaN
1,3,2,None,2.0,NaN
2,4,3,None,3.0,NaN
3,5,4,aa16d5bb51857f83b96796f523e3567fb8241561659955...,NaN,NaN
4,6,5,1bf5e2e6edf841ea51488a1a822f5539038254ed0d979c...,NaN,NaN


##### Load Menu Dataframe
____

In [170]:
menu = pd.read_sql_table(table_name='menu', con=disk_engine)

menu.head(5)

,id,restaurant_id,title,content,enabled
0,1,1,Combo 1,Super Combo 1,1
1,2,1,Combo 2,Super Combo 2,1
2,3,1,Combo 3,Super Combo 3,1


##### Load Menu Items DataFrame
____

In [171]:
menu_items = pd.read_sql_table(table_name='menu_items', con=disk_engine)

menu_items.head(5)

,id,menu_id,ingredient_id
0,1,1,4
1,2,1,3
2,3,2,6
3,4,2,8
4,5,3,5


##### Load Ingredients DataFrame
____

In [172]:
ingredients = pd.read_sql_table(table_name='ingredients', con=disk_engine)

ingredients.head(5)

,id,allergen,diet_adherence_tag,name,price,size,calories,promotion_tag,unit
0,2,None,None,baked potato,3.00,400.0,100,None,g
1,3,None,None,broccoli,1.00,50.0,30,None,g
2,4,None,None,salmon,12.99,600.0,400,None,g
3,5,None,None,fried chicken,7.99,400.0,400,None,g
4,6,None,None,steak,16.99,800.0,800,None,g


##### Helper Functions
____

In [173]:
def get_payment_method(userid):
    cond = payment_method['userid'] == userid
    
    return int(payment_method[cond]['id'].values[0])

def get_customer_address(userid):
    cond = credentials['userid'] == userid    
    customer = int(credentials[cond]['customer_id'].values[0])
    
    cond = customers['id'] == customer    
    return int(customers[cond]['address'].values[0])    

def get_menu_price_total(menu_item):
    cond = menu_items['menu_id'] == menu_item
    
    return float(pd.merge(left=menu_items[cond], right=ingredients, how='inner', left_on='ingredient_id', right_on='id')['price'].sum())   

##### Add Order
____

In [175]:
total = 1000
reviews = []

for i in range(1, total+1):
    userid     = int(credentials['userid'].sample(n=1).values[0])
    driver     = int(drivers['id'].sample(n=1).values[0])
    payment    = get_payment_method(userid)
    deliver_to = get_customer_address(userid)
    created    = (datetime.datetime.now() + datetime.timedelta(hours=random.randint(1,720)))
    status     = 'created'
    
    sql = "INSERT INTO `order`(userid, driver, payment_method, deliver_to, created, status) VALUES (%s, %s, %s, %s, %s, %s)"
    values = (userid, driver, payment, deliver_to, created.strftime('%Y-%m-%d %H:%M:%S'), status)

    order_id = disk_engine.execute(sql, values).lastrowid
    
    reviews.append({
        'order'  : order_id,
        'driver' : driver
    })
    
    # order item
    menu_item = int(menu['id'].sample(n=1).values[0])
    quantity  = random.randint(1, 5)
    total     = get_menu_price_total(menu_item) * quantity
    
    sql = "INSERT INTO `order_item`(order_id, menu_item, quantity, total) VALUES (%s, %s, %s, %s)"
    values = (order_id, menu_item, quantity, total)
    
    disk_engine.execute(sql, values)
    
    # update order header
    order_choice = random.randint(1, 4)
    
    if (order_choice == 1): 
        # order on queue
        pass
    
    elif (order_choice == 2):
        status      = 'in-progress'
        
        sql = "UPDATE `order` SET status= %s WHERE id = %s"
        values = (status, order_id)
        
        disk_engine.execute(sql, values)
        
    elif (order_choice == 3):
        delivered   = (created + datetime.timedelta(minutes=random.randint(1,90))).strftime('%Y-%m-%d %H:%M:%S')
        status      = 'delivered'
        tip         = random.uniform(0.05, 0.20)
        order_total = (total * tip) + total
        
        sql = "UPDATE `order` SET delivered = %s, status= %s,  tip = %s, total = %s WHERE id = %s"
        values = (delivered, status, tip, order_total, order_id)
        
        disk_engine.execute(sql, values)
    else:
        canceled   = (datetime.datetime.now() + datetime.timedelta(minutes=random.randint(1,90))).strftime('%Y-%m-%d %H:%M:%S')
        status      = 'canceled'
        tip         = random.uniform(0.05, 0.20)
        order_total = (total * tip) + total
        
        sql = "UPDATE `order` SET canceled = %s, status= %s,  tip = %s, total = %s WHERE id = %s"
        values = (canceled, status, tip, order_total, order_id)      
        
        disk_engine.execute(sql, values)


##### Order Review
____

In [176]:
for review in reviews:
    review_choice = random.randint(0, 1)
    
    if (review_choice == 1):
        rate = random.randint(1, 5)
        comments = "Autogenerated review"
        
        sql = "INSERT INTO `order_review`(order_id, rate, comments) VALUES (%s, %s, %s)"
        values = (review['order'], rate, comments)
    
        disk_engine.execute(sql, values)       
        

##### Delivery Review
____

In [177]:
for review in reviews:
    review_choice = random.randint(0, 1)
    
    if (review_choice == 1):
        rate = random.randint(1, 5)
        comments = "Autogenerated review"
        
        sql = "INSERT INTO `delivery_review`(order_id, rate, comments) VALUES (%s, %s, %s)"
        values = (review['order'], rate, comments)
    
        disk_engine.execute(sql, values)       